In [1]:
from urllib.request import urlopen as uReq
from bs4 import BeautifulSoup as soup

import numpy as np
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt
%matplotlib inline


In [2]:
if 1==0:
    desc_list = []
    name_list = []
    cat_list = []

    for i in range(5100,5500):
        base_url = 'http://www.seret.co.il/movies/s_movies.asp?MID='+str(i)

        uclient = uReq(base_url)
        html_page = uclient.read()

        uclient.close()
        page_soup = soup(html_page,'html.parser')

        review = page_soup.findAll("span",{'itemprop':"description"})
        name = page_soup.find_all('title')


        try:
            temp1 = review[0].get_text()
            temp2 = str(name).split('|')[0].split('>')[1].strip()
            categories = page_soup.find_all('a',class_='DarkGreenU12')
            temp = [t for t in categories if 'GID' in str(t)]
            cat = [str(c).split('GID=')[1].split('\"')[0] for c in temp]
            desc_list.append(temp1)
            name_list.append(temp2)
            cat_list.append(cat)
        except:
            pass
    
    df = pd.DataFrame({'names':name_list,'descriptions':desc_list,'categories':cat_list})
    df.to_csv('movie_descriptions_and_categories.csv',index=False)

In [3]:
df = pd.read_csv('movie_descriptions_and_categories.csv')

In [4]:
df.descriptions[3]

'מגי היא אם חד-הורית מניו-יורק המתרגלת לחייה החדשים כרווקה. בניסיון להתקרב לביתה סאמר, נערה מתבגרת ומרדנית, היא מחליטה לצאת עימה לטיול בכפר באיטליה, בו ביקרה פעמים רבות כשהייתה צעירה. בכפר פוגשת מגי את לוקה, אהובה מלפני עשרים שנים. בזמן שהשניים מגששים את ד ...\xa0\xa0המשך ...'

In [5]:
base_url = 'http://www.seret.co.il/movies/s_movies.asp?MID=5555'

uclient = uReq(base_url)
html_page = uclient.read()

uclient.close()
page_soup = soup(html_page,'html.parser')

review = page_soup.findAll("span",{'itemprop':"description"})
name = page_soup.find_all('title')
categories = page_soup.find_all('a')
temp = [t for t in categories if 'GID' in str(t)]
cat = [str(c).split('GID=')[1].split('\"')[0] for c in temp]

In [6]:
cat_desc = [str(cd).split('\">')[1].split('<')[0] for cd in temp]

conversion_df = pd.DataFrame({'cat_id':cat,'cat_desc':cat_desc})
conversion_df

,cat_desc,cat_id
0,קומדיה,8
1,מתח / פעולה,7
2,דרמה,4
3,ילדים,2
4,אימה,1
5,אנימציה,16
6,הרפתקאות,15
7,זר,13
8,ישראלי,11
9,מדע בדיוני,3


In [7]:
def has_category(x,c):
    return int(str(c) in x)
df1 = df.copy()
for i in range(1,17):
    df1['has_cat_'+str(i)] = df.categories.apply(lambda x: has_category(x,i))



In [8]:
df1.iloc[:,4:].sum()

has_cat_2      77
has_cat_3      99
has_cat_4     157
has_cat_5      77
has_cat_6      69
has_cat_7     147
has_cat_8      90
has_cat_9      27
has_cat_10     11
has_cat_11     41
has_cat_12      0
has_cat_13     42
has_cat_14     15
has_cat_15     77
has_cat_16     61
dtype: int64

In [10]:
df1['selected'] = (df1['has_cat_2'] | df1['has_cat_3'] | df1['has_cat_7'] | df1['has_cat_8'] | df1['has_cat_4'] )

In [11]:
df = df[df1.selected==1]

In [12]:
del df1

In [13]:
from sklearn.feature_extraction.text import CountVectorizer
cvec = CountVectorizer()

In [14]:
X_counts = cvec.fit_transform(df.descriptions)

In [15]:
X_counts.shape

(380, 6146)

In [16]:
type(X_counts)

scipy.sparse.csr.csr_matrix

In [17]:
len(cvec.vocabulary_)

6146

In [18]:
from sklearn.cluster import KMeans
km = KMeans(n_clusters=5,n_jobs=4,random_state=2016,max_iter=1000)
clst = km.fit_predict(X_counts)

In [20]:
df.categories

0       ['7', '15', '4']
2             ['4', '6']
3             ['8', '9']
4        ['7', '4', '6']
6      ['16', '2', '15']
7       ['16', '2', '8']
8            ['16', '2']
9            ['4', '13']
10                 ['4']
11           ['4', '17']
12           ['7', '15']
13      ['6', '4', '17']
14                 ['7']
15            ['3', '7']
16           ['3', '15']
17            ['7', '4']
18            ['8', '9']
21            ['7', '4']
22       ['3', '7', '4']
23      ['16', '2', '8']
24           ['2', '16']
25           ['4', '13']
26      ['14', '8', '4']
27           ['15', '4']
28      ['7', '15', '4']
29     ['2', '11', '15']
30      ['8', '9', '11']
31            ['7', '4']
32           ['7', '15']
33           ['15', '4']
             ...        
368     ['8', '4', '13']
369          ['4', '17']
370           ['7', '4']
371    ['16', '14', '8']
372    ['16', '14', '8']
373                ['4']
374          ['15', '7']
375           ['4', '3']
376           ['8', '4']


In [21]:
df['clst'] = clst
df['first_cat'] = df.categories.apply(lambda x: int(x[0]))

ValueError: invalid literal for int() with base 10: '['

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_pred=df.clst,y_true=df.first_cat)
sns.heatmap(cm)

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer,TfidfVectorizer

vectorizer = TfidfVectorizer(max_df=0.4, max_features=500,min_df=2,use_idf=False)

X_tfidf = vectorizer.fit_transform(df.descriptions)

In [ ]:
X_tfidf
km = KMeans(n_clusters=13,n_jobs=4,random_state=2016,max_iter=1000)
clst = km.fit_predict(X_tfidf)
df['clst'] = clst
cm = confusion_matrix(y_pred=df.clst,y_true=df.first_cat)
sns.heatmap(cm)

In [ ]:
from gensim.corpora import Dictionary, MmCorpus
from gensim.models.ldamulticore import LdaMulticore

import pyLDAvis
import pyLDAvis.gensim
import warnings

In [ ]:
dictionary = X_counts.get_feature_names()

In [ ]:
lda = LdaMulticore(X_tfidf,num_topics=15,workers=4)